In [1]:
import pandas as pd
from EDA.OutlierHandling import OutlierHandling
from EDA.Encoding import Encoding
from EDA.NullHandling import NullHandling
from EDA.ColumnTypeIdentification import ColumnTypeIdentification
from EDA.FeatureReduction import FeatureReduction
import warnings
warnings.filterwarnings("ignore")
import threading
from Modelling.Classification import Classification
from sklearn.metrics import accuracy_score,f1_score
from itertools import product
from pathlib import Path

In [2]:
from MLAccelerator import MLAccelerator

In [3]:
from flask import Flask, render_template, request, redirect, url_for
import pandas as pd
app = Flask(__name__)

# render webpage
@app.route('/')
def home():
    return render_template('Home.html')


@app.route('/', methods=['POST', 'GET'])
def get_data():
    if request.method == 'POST':
        flag_list ={}
        final_list={}
        metric_dict={'accuracy_score':accuracy_score,'f1_score':f1_score}
        flag_list['outlier']=request.form.getlist('outlier')
        flag_list['nullhandle']=request.form.getlist('nullhandle')
        flag_list['feature']=request.form.getlist('feature')
        flag_list['encoding']=request.form.getlist('encoding')
        flag_list['metric']=request.form.getlist('metric')
        flag_list['model']=request.form.getlist('model')
        for key in flag_list:
            if not flag_list[key]:
                final_list[key] = [None]
            else:
                final_list[key] = flag_list[key]
        df = pd.read_csv(request.files.get('file'),delimiter=',')      
        y = request.form['target']
        ml=MLAccelerator(df,y,final_list,metric_dict)
        result=ml.execute()
        return result
    

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/May/2020 16:35:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 16:35:39] "GET /Home_files/font-awesome.min.css HTTP/1.1" 404 -


[<function accuracy_score at 0x0000022942585268>]
Running Thread: 0
100%|█████████████████████████████████████████████| 100/100 [00:16<00:00,  5.66trial/s, best loss: -0.8715083798882681]


127.0.0.1 - - [18/May/2020 16:37:33] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2020 16:38:13] "GET / HTTP/1.1" 200 -


In [ ]:
            
        dispatcher = {'accuracy_score': accuracy_score}
        print(dispatcher[final_list['metric']])

In [ ]:
        flag_list = {}
            for key in final_list:
            if not final_list[key]:
                flag_list[key] = False
            else:
                flag_list[key] = True

In [ ]:
#outlier_list = request.form.getlist('outlier')
#nullhandle_list = request.form.getlist('nullhandle')
#feature_list = request.form.getlist('feature')
#encoding_list = request.form.getlist('encoding')
#metric_list = request.form.getlist('metric')